In [1]:
import mlbeats
import pandas as pd
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
import biosppy

In [2]:
filenumber=[100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 200, 201, 202, 203, 205, 207, 208, 209, 210, 212, 213, 214, 215, 217, 219, 220, 221, 222, 223, 228, 230, 231, 232, 233, 234]

In [3]:
heart_beat=pd.DataFrame()

In [4]:
for j in filenumber:
    test_file = pd.read_csv("mitbih_database/"+str(j)+".csv")
    ecg_signal = test_file.iloc[:, 1].to_numpy()
    data = []
    with open("mitbih_database/"+str(j)+"annotations.txt", 'r') as data_file:
        for line in data_file:
            item=line.split()
            if len(item)==7:
                item[2]=item[-1]
                item=item[0:7]
                data.append(item[1:3])
            else:
                data.append(item[1:3])
    annotations=pd.DataFrame(data[2:], columns=["Sample#", "Result"])
    rpeak=[]
    for i in range(len(annotations['Sample#'])):
        x = int(annotations['Sample#'][i])
        rpeak.append(x)
    rpeak = np.array(rpeak)
    epochs, sudo_rpeak=biosppy.signals.ecg.extract_heartbeats(signal=ecg_signal, rpeaks=rpeak, sampling_rate=360, before=0.4, after=0.4)
    
    
    
    
    labels = []
    for k in sudo_rpeak:
        row = annotations[annotations["Sample#"] == str(k)]
        labels.append(row.iloc[:, 1].to_numpy()[0])
    S = {"A", "a", "J", "S"}
    N = {"N", "L", "R", "e", "j"}
    V = {"V", "E"}
    F = {"F"}
    Q = {"/", "f", "Q"}
    for m in range(len(labels)):
        if labels[m] in N:
            labels[m] = "N"
        elif labels[m] in S:
            labels[m] = "S"
        elif labels[m] in V:
            labels[m] = "V"
        elif labels[m] in F:
            labels[m] = "F"
        elif labels[m] in Q:
            labels[m] = "Q"
        else:
            pass

        
      
        
        
    drop_num=[] 
    for n in range(288):
        if n%3 != 0:
            drop_num.append(n)
            
            
        
    epochs = pd.DataFrame(epochs)
    epochs["Result"] = labels
    epochs = epochs.drop(drop_num, axis=1)
    heart_beat = pd.concat([heart_beat, epochs], axis=0)
    print(j)

100
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
121
122
123
124
200
201
202
203
205
207
208
209
210
212
213
214
215
217
219
220
221
222
223
228
230
231
232
233
234


In [7]:
print(heart_beat)

        0    3    6    9   12   15   18   21   24   27  ...  261  264  267  \
0     968  970  968  969  967  969  964  965  962  967  ...  959  958  962   
1     966  960  961  958  961  956  958  954  958  955  ...  962  964  962   
2     959  960  954  957  956  954  953  954  954  954  ...  970  965  968   
3     964  962  964  956  960  957  960  955  958  954  ...  960  963  960   
4     963  964  959  961  958  959  957  957  961  958  ...  965  966  967   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2757  972  984  982  981  980  984  978  976  973  975  ...  977  975  981   
2758  993  991  988  980  983  982  977  975  981  980  ...  971  972  970   
2759  984  974  970  971  972  970  965  967  970  972  ...  970  978  976   
2760  980  980  979  984  977  975  972  978  973  969  ...  982  984  979   
2761  987  986  985  989  994  993  988  983  985  982  ...  964  966  962   

      270  273  276  279  282  285  Result  
0     960  963  96

In [8]:
heart_beat.to_csv("beat_extraction.csv")